# Application of notebooks in digital twins

This use case will demonstrate how data relevant to various digital twins models can be supported by notebooks to provide examples of use.

This approach employs a workflow as shown in the following picture.

<img src="imgs/workflow.png" width="100%">

### Available Datasets 

This section describes existing collections of maps made available by relevant institutions.

For instance, [Europeana](https://www.europeana.eu/en/galleries/15694-twin-it-a-pan-european-collection-of-heritage-3-d-models) provides access to high-quality 3D models accross Europe. For example, the [Reconstruction of the Heidentor](https://www.europeana.eu/en/item/1113/heidentor_02), a triumphal monument known as Heidentor, built as a four-gate arched monument: 

<img width="40%" src="https://api.europeana.eu/thumbnail/v3/400/d0410ffde56e6a5553d5089b034460f8">

The [Sketchfab](https://sketchfab.com/categories/cultural-heritage-history) also provides access to a collection Cultural Heritage 3D models. Other initiatives are related to data spaces from the [CH sector combined with 3D models](https://www.egi.eu/article/eureka3d-supporting-3d-in-the-data-space-for-cultural-heritage/). Organizations like UNESCO have created a list of [“World Heritage Sites in Danger”](https://whc.unesco.org/en/danger/) in order to prioritize their conservation.

## We will use the Europeana as example

### Retrieve Metadata

Europeana provides access to its API in order to retrieve json as a format. In order to do so, we need to request [an API key](https://pro.europeana.eu/page/get-api). You need to add the API key in the following URL:

In [60]:
url = 'https://api.europeana.eu/record/1113/heidentor_02.json?wskey=MYKEY'

The history saving thread hit an unexpected error (OperationalError('attempt to write a readonly database')).History will not be written to the database.


In [61]:
import requests

resp = requests.get(url=url)
#print (resp.text) ## uncomment to see the content

with open("data/europeana/metadata-europeana.json", "w") as text_file:
    text_file.write(resp.json)

#### Extraction

Now we transform the human-readable text to a CSV so the data can be analysed and adapted easily. For this, we create a CSV file, each row containing a record.

### Transform the metadata to RDF

In this step we transform the human-readable metadata to machine-readable by using [Resource Description Framework (RDF)](https://www.w3.org/TR/rdf12-concepts/), a standard to publish data in the form of triplets promoted by the W3C.

Following other approaches and previous work, the python library RDFLib is employed to create the RDF data by means of Schema.org as main vocabulary.

Note that as an example we use the National Library of Spain. Additional datasets could be used following a similar approach, in order to create a larger dataset with data integrated from several institutions.

In [15]:
from rdflib import Graph, URIRef, Literal, Namespace
from rdflib.namespace import FOAF, RDF, RDFS, DCTERMS, VOID, DC, SKOS, OWL, XSD
import datetime

g = Graph()
g.bind("foaf", FOAF)
g.bind("rdf", RDF)
g.bind("rdfs", RDFS)
g.bind("dcterms", DCTERMS)
g.bind("dc", DC)
g.bind("void", VOID)
g.bind("skos", SKOS)
g.bind("owl", OWL)

schema = Namespace("https://schema.org/")
g.bind("schema", schema)

dcat = Namespace("http://www.w3.org/ns/dcat#")
g.bind("dcat", dcat)

wd = Namespace("http://www.wikidata.org/entity/")
g.bind("wd", wd)

domain = 'https://example.org/'
domainLanguage = domain + 'map/'

##### We add the metadata of the dataset

In [16]:
# First, we create all the required URIS
catalog = URIRef(domain + "catalog/digital-twins")
europeana = URIRef(domain + "organization/europeana")
dataset = URIRef(domain + "dataset/digital-twins")
digital_twins_csv = URIRef(domain + "distribution/digital-twins-json")
digital_twins_ttl = URIRef(domain + "distribution/digital-twins-ttl")

In [17]:
# We describe the dataset
g.add((catalog, RDF.type, schema.Dataset))
g.add((catalog, RDF.type, dcat.catalog))
g.add((catalog, RDFS.label, Literal("The Reconstruction of the Heidentor - 3D model Europeana")))
g.add((catalog, schema.url, URIRef("https://bnedigital.bne.es/")))
g.add((catalog, FOAF.homepage, URIRef("https://bnedigital.bne.es/")))
g.add((catalog, schema.description, Literal("The Reconstruction of the Heidentor - 3D model Europeana")))
g.add((catalog, schema.name, Literal("The Reconstruction of the Heidentor - 3D model Europeana")))
g.add((catalog, DCTERMS.title, Literal("The Reconstruction of the Heidentor - 3D model Europeana")))
g.add((catalog, DCTERMS.publisher, URIRef(europeana))) # relation dataset-publisher
g.add((catalog, DC.title, Literal("Europeana")))
g.add((catalog, schema.license, URIRef('http://creativecommons.org/licenses/by-nc/4.0/')))

now = datetime.datetime.now()
g.add((catalog, schema.dateCreated, Literal(str(now)[:10])))

<Graph identifier=Ne8c3760cc88544b3a0b3abc64fab124e (<class 'rdflib.graph.Graph'>)>

In [18]:
# We describe the BNE
g.add((europeana, RDF.type, FOAF.Organization))
g.add((europeana, RDFS.label, Literal("Europeana")))
g.add((europeana, FOAF.homepage, URIRef("https://www.europeana.eu/")))

<Graph identifier=Ne8c3760cc88544b3a0b3abc64fab124e (<class 'rdflib.graph.Graph'>)>

In [19]:
# We describe the dataset
g.add((dataset, RDF.type, dcat.Dataset))
g.add((dataset, DCTERMS.title, Literal("The Reconstruction of the Heidentor - 3D model Europeana", lang="en")))
g.add((dataset, dcat.keyword, Literal("3D models")))
g.add((dataset, dcat.keyword, Literal("Metadata")))
g.add((dataset, dcat.keyword, Literal("Collections as data")))
g.add((dataset, DCTERMS.issued, Literal(str(now)[:10])))
g.add((dataset, DCTERMS.language, URIRef("http://id.loc.gov/vocabulary/iso639-1/en")))
g.add((dataset, dcat.distribution, URIRef(digital_twins_json)))

<Graph identifier=Ne8c3760cc88544b3a0b3abc64fab124e (<class 'rdflib.graph.Graph'>)>

In [20]:
# We describe the distribution TTL 
g.add((digital_twins_ttl, RDF.type, dcat.Distribution))
g.add((digital_twins_ttl, dcat.downloadURL , URIRef("https://raw.githubusercontent.com/hibernator11/eccch-use-cases/refs/heads/main/data/maps-bne/dataset_bne.ttl")))
g.add((digital_twins_ttl, DCTERMS.title, Literal("TTL distribution of the 3D model The Reconstruction of the Heidentor", lang="en")))
g.add((digital_twins_ttl, DCTERMS.title, Literal("Distribución en TTL del modelo 3D Reconstruction of the Heidentor de Europeana", lang="es")))
g.add((digital_twins_ttl, dcat.mediaType, URIRef("http://www.iana.org/assignments/media-types/application/n-triples")))
g.add((digital_twins_ttl, dcat.byteSize, Literal('528000', datatype=XSD.integer)))

<Graph identifier=Ne8c3760cc88544b3a0b3abc64fab124e (<class 'rdflib.graph.Graph'>)>

### Enriching the 3D model

This work employs previous work concerning the description of reactivity of digital twins for CH by including the semantic description of sensors and activators and all of the process of interacting with the real world. In particular, is based on several CRM extensions such as the Heritage Digital Twins Ontology (HDTO), CRMdig and CRMsci.

A sensor can be defined in general terms as a digital device placed on physical objects or in specific locations intended to measure and collect data about them, process it, and transmit it to the digital twin system for analysis and further processing. In this work, we could imagine that the physical object Heidentor (identified in Wikidata with https://www.wikidata.org/wiki/Q1594216), has several sensors measuring physical or environmental properties, such as temperature, pressure, humidity, light, sound or position and can be of different types, such as analogic or digital, wired or wireless.

A class Sensor (e.g., subclass of the class D8 Digital Device of CRMdig), a class to describe instances of material items capable of
processing or producing digital data. The location of a resource typed as Sensor can be described using a property "is positioned on". The status of a Sensor can be described using the CRM class "E3 Condition State", reporting sensor states and/or errors. Sensors use software to configure, calibrate and monitor which is represented by the class D14 Software class of CRMdig.

Based on CRM, the measurement operations performed by sensors can be modelled as CRM events.

Please, see the following reference for more information about the data modelling:

- Niccolucci, F., & Felicetti, A. (2024). Digital Twin Sensors in Cultural Heritage Ontology Applications. Sensors, 24(12), 3978. https://doi.org/10.3390/s24123978

In [ ]:
### To be done
# Data modelling of digital twins combined with sensors

#### Now we can query the graph using SPARQL

In [ ]:
print('##### Properties:')

# Query the data in g using SPARQL
q = """
    SELECT distinct ?prop
    WHERE {
        ?s ?prop ?o .
    }
"""

# Apply the query to the graph and iterate through results
for r in g.query(q):
    print(r["prop"])

As an example we can retrieve the metadata of our dataset

In [ ]:
print('##### Dataset information:')

# Query the data in g using SPARQL
q = """
    PREFIX schema: <https://schema.org/>
    SELECT distinct ?p ?o
    WHERE {
        ?s a schema:Dataset .
        ?s ?p ?o
    }
"""

# Apply the query to the graph and iterate through results
for r in g.query(q):
    print(r["p"] + ": " + r["o"])

### Integration with the ECCCH

[ECHOES](https://www.echoes-eccch.eu/faq/) is building a federated Knowledge Graph to allow for high level integration of resources. It will also serve as an entry point for all queries and requests related to any kind of information available within the Cultural heritage Cloud. The Knowledge Graph will use the proposed Heritage Digital Twin Ontology (HDTO) to unify descriptions and facilitate query and navigation. The current version of the ECHOES HDTO is available [here](https://www.echoes-eccch.eu/wp-content/uploads/2025/06/ECHOES_HDT_Ontology.pdf). The main vocabulary employed to describe the resources is [CIDOC-CRM](https://cidoc-crm.org/).

The following illustration shows how we modelled the outputs of this work in order to be integrated into the ECCCH.
<img width="80%" src="imgs/eccch-integration-steps.png">

In [1]:
## To Be done!
# Integration with the ECCCH once the API and fina data model is available

### Publication & dissemination

This step involves the publication of the results obtained including the dataset and this notebook in different platforms such as the Social Sciences and Humanities Open Marketplace and Zenodo.


As an example, we will use the sandbox service of Zenodo. Note that if you want to use this code for production purposes, it is required to update the URL. First, we need to create an access token in this [link](https://zenodo.org/account/settings/applications/tokens/new/). Note that we also need a token for the sandbox Zenodo.

In [43]:
# https://developers.zenodo.org/
import requests
ACCESS_TOKEN = 'ChangeMe'

headers = {
    "Content-Type": "application/json",
    "Authorization": f"Bearer {ACCESS_TOKEN}"
}
r = requests.post('https://sandbox.zenodo.org/api/deposit/depositions',
                   json={},
                   headers=headers)
r.status_code
r.json()

{'created': '2026-01-12T17:41:07.632652+00:00',
 'modified': '2026-01-12T17:41:07.740296+00:00',
 'id': 424692,
 'conceptrecid': '424691',
 'metadata': {'access_right': 'open',
  'prereserve_doi': {'doi': '10.5281/zenodo.424692', 'recid': 424692}},
 'title': '',
 'links': {'self': 'https://sandbox.zenodo.org/api/deposit/depositions/424692',
  'html': 'https://sandbox.zenodo.org/deposit/424692',
  'badge': 'https://sandbox.zenodo.org/badge/doi/.svg',
  'files': 'https://sandbox.zenodo.org/api/deposit/depositions/424692/files',
  'bucket': 'https://sandbox.zenodo.org/api/files/a40e829f-dd24-4e7b-b228-7c4e048a270c',
  'latest_draft': 'https://sandbox.zenodo.org/api/deposit/depositions/424692',
  'latest_draft_html': 'https://sandbox.zenodo.org/deposit/424692',
  'publish': 'https://sandbox.zenodo.org/api/deposit/depositions/424692/actions/publish',
  'edit': 'https://sandbox.zenodo.org/api/deposit/depositions/424692/actions/edit',
  'discard': 'https://sandbox.zenodo.org/api/deposit/depos

Now, let’s upload a new file:

In [44]:
bucket_url = r.json()["links"]["bucket"]
deposition_id = r.json()["id"]

First, we create a zip file with the notebook and the requirements.txt file:

In [45]:
from zipfile import ZipFile

# List of files to include in the archive
file_list = ["Digital-Twins.ipynb", "requirements.txt"]

# Create ZIP file and write files into it
with ZipFile("output.zip", "w") as zipf:
   for file in file_list:
      zipf.write(file)

Then, we call the API:

In [47]:
filename = "output.zip"
path = "%s" % filename
headers = {'Authorization': f'Bearer {ACCESS_TOKEN}'}

''' 
The target URL is a combination of the bucket link with the desired filename
seperated by a slash.
'''
with open(path, "rb") as fp:
    r = requests.put(
        "%s/%s" % (bucket_url, filename),
        data=fp,
        headers=headers,
    )
r.json()

{'created': '2026-01-12T17:41:16.912746+00:00',
 'updated': '2026-01-12T17:41:17.052565+00:00',
 'version_id': 'f8cd6099-25aa-4105-aa41-91921a295466',
 'key': 'output.zip',
 'size': 28194,
 'mimetype': 'application/zip',
 'checksum': 'md5:4df29214004c115c3e7a0ab4a433da97',
 'is_head': True,
 'delete_marker': False,
 'links': {'self': 'https://sandbox.zenodo.org/api/files/a40e829f-dd24-4e7b-b228-7c4e048a270c/output.zip',
  'version': 'https://sandbox.zenodo.org/api/files/a40e829f-dd24-4e7b-b228-7c4e048a270c/output.zip?version_id=f8cd6099-25aa-4105-aa41-91921a295466',
  'uploads': 'https://sandbox.zenodo.org/api/files/a40e829f-dd24-4e7b-b228-7c4e048a270c/output.zip?uploads=1'}}

We can also add metadata to the record:

In [49]:
data = {
     'metadata': {
         'title': 'Application of notebooks in digital twins',
         'upload_type': 'software',
         'description': 'This use case will demonstrate how data relevant to various digital twins models can be supported by notebooks to provide examples of use.',
         'creators': [{'name': 'Candela, Gustavo',
                       'affiliation': 'University of Alicante'}]
     }
 }
headers = {
    'Content-Type': 'application/json',
    'Authorization': f'Bearer {ACCESS_TOKEN}'
}
r = requests.put('https://sandbox.zenodo.org/api/deposit/depositions/%s' % deposition_id,
                  data=json.dumps(data),
                  headers=headers)
r.status_code

200

The last step is the publication:

In [50]:
headers = {'Authorization': f'Bearer {ACCESS_TOKEN}'}
r = requests.post('https://sandbox.zenodo.org/api/deposit/depositions/%s/actions/publish' % deposition_id,
                      headers=headers)
r.status_code
# 202

202

And now we can see the result in Zenodo:

<img src="imgs/zenodo-publication.png" width="70%">

We can reproduce the same with additional platforms such as [Wikidata](https://www.wikidata.org/) and the [Social Sciences and Humanities Open Marketplace](https://marketplace.sshopencloud.eu/about/api-documentation)

In the particular case of Wikidata, existing [python libraries](https://www.mediawiki.org/wiki/Manual:Pywikibot/Wikidata) can be used to extract and create entities.

### References

- Candela, G., Rosiński, C., & Margraf, A. (2025). A reproducible framework to publish and reuse Collections as data: the case of the European Literary Bibliography (Version 4, Vol. 965, Issue 170). Transformations: A DARIAH Journal . https://doi.org/10.46298/transformations.14729
- Gustavo Candela, Javier Pereda, Dolores Sáez, Pilar Escobar, Alexander Sánchez, Andrés Villa Torres, Albert A. Palacios, Kelly McDonough, and Patricia Murrieta-Flores. 2023. An Ontological Approach for Unlocking the Colonial Archive. J. Comput. Cult. Herit. 16, 4, Article 74 (December 2023), 18 pages. https://doi.org/10.1145/3594727
- Niccolucci F, Markhoff B, Theodoridou M et al. The Heritage Digital Twin: a bicycle made for two. The integration of digital methodologies into cultural heritage research [version 1; peer review: 2 approved with reservations]. Open Res Europe 2023, 3:64 (https://doi.org/10.12688/openreseurope.15496.1)
- https://developers.zenodo.org/#quickstart-upload
- https://www.echoes-eccch.eu/wp-content/uploads/2025/06/ECHOES_HDT_Ontology.pdf
- https://marketplace.sshopencloud.eu/about/api-documentation
- https://www.wikidata.org/
- https://cidoc-crm.org/sites/default/files/CRMdigv4.0.pdf
- https://arxiv.org/html/2503.02167v1
- https://digital-strategy.ec.europa.eu/en/library/basic-principles-and-tips-3d-digitisation-cultural-heritage